In [1]:
from huggingface_hub import hf_hub_download
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "Elriggs/pythia-70m-deduped"

# The layer around which we want to interpret by looking at the last layer
layer_focus = 1
assert layer_focus > 0

In [2]:
def load_autoencoder(layer: int):
	ae_download_location_main = hf_hub_download(repo_id=model_id, filename=f"tied_residual_l{layer_focus}_r6/_63/learned_dicts.pt")
	all_autoencoders = torch.load(ae_download_location_main)
	all_l1s = [hyperparams["l1_alpha"] for autoencoder, hyperparams in all_autoencoders]
	# TODO: choose best one???
	print(all_l1s)
	auto_num = 5
	autoencoder, hyperparams = all_autoencoders[auto_num]
	# You want a hyperparam around 1e-3. Higher is less features/datapoint (at the cost of reconstruction error); lower is more features/datapoint (at the cost of polysemanticity)
	autoencoder.to_device(device)
	print(hyperparams)
	return autoencoder, hyperparams

autoencoder_main, hyperparams_main = load_autoencoder(layer_focus)
autoencoder_prior, hyperparams_prior = load_autoencoder(layer_focus - 1)

[0.0, 9.999999747378752e-05, 0.00019306977628730237, 0.000372759357560426, 0.0007196856895461679, 0.0013894954463467002, 0.0026826958637684584, 0.005179474595934153, 0.009999999776482582]
{'dict_size': 3072, 'l1_alpha': 0.0013894954463467002}
[0.0, 9.999999747378752e-05, 0.00019306977628730237, 0.000372759357560426, 0.0007196856895461679, 0.0013894954463467002, 0.0026826958637684584, 0.005179474595934153, 0.009999999776482582]
{'dict_size': 3072, 'l1_alpha': 0.0013894954463467002}


In [3]:
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from transformer_lens import HookedTransformer
setting = "residual"
model_name = "EleutherAI/pythia-70m-deduped"

model = HookedTransformer.from_pretrained(model_name, device=device)

def get_cache_name_neurons(layer: int):
    if setting == "residual":
        cache_name = f"blocks.{layer}.hook_resid_post"
        neurons = model.cfg.d_model
    elif setting == "mlp":
        cache_name = f"blocks.{layer}.mlp.hook_post"
        neurons = model.cfg.d_mlp
    elif setting == "attention":
        cache_name = f"blocks.{layer}.hook_attn_out"
        neurons = model.cfg.d_model
    elif setting == "mlp_out":
        cache_name = f"blocks.{layer}.hook_mlp_out"
        neurons = model.cfg.d_model
    else:
        raise NotImplementedError
    return cache_name, neurons

cache_name, neurons  = get_cache_name_neurons(layer_focus)
cache_name_prior, _  = get_cache_name_neurons(layer_focus - 1)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m-deduped into HookedTransformer


In [15]:
# Downnload dataset
from datasets import Dataset, load_dataset
# dataset_name = "NeelNanda/pile-10k"
# dataset_name = "NeelNanda/pile-10k"
dataset_name = "JeanKaddour/minipile"
token_amount= 40
#TODO: change train[:1000] to train if you want whole dataset
# 100_000 datasets
dataset = load_dataset(dataset_name, split="train[:100000]").map(
    lambda x: model.tokenizer(x['text']),
    batched=True,
).filter(
    lambda x: len(x['input_ids']) > token_amount
).map(
    lambda x: {'input_ids': x['input_ids'][:token_amount]}
)
# TODO: we can maybe make this faster for the larger dataset?

Found cached dataset parquet (/home/lev/.cache/huggingface/datasets/JeanKaddour___parquet/JeanKaddour--minipile-0d7d2d1ff79d1d36/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /home/lev/.cache/huggingface/datasets/JeanKaddour___parquet/JeanKaddour--minipile-0d7d2d1ff79d1d36/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-33be347e89413cee.arrow
Loading cached processed dataset at /home/lev/.cache/huggingface/datasets/JeanKaddour___parquet/JeanKaddour--minipile-0d7d2d1ff79d1d36/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-f6085a72c3f9dd17.arrow
Loading cached processed dataset at /home/lev/.cache/huggingface/datasets/JeanKaddour___parquet/JeanKaddour--minipile-0d7d2d1ff79d1d36/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-904da2d039ac9ca3.arrow


# Get Dictionary Activations

In [17]:
# TODO: in chunks...
# Now we can use the model to get the activations
from torch.utils.data import DataLoader
from datasets import DatasetDict
from tqdm.auto import tqdm
from einops import rearrange
import math

# MAX_CHUNK_SIZE = 1_000

def get_activations(autoencoder):
    num_features, d_model = autoencoder.encoder.shape
    datapoints = dataset.num_rows
    dictionary_activations_final = np.memmap('dict.mymemmap', dtype='float32', mode='w+', shape=(datapoints * token_amount, num_features))
    # neuron_activations = torch.zeros((datapoints*token_amount, d_model))
    neuron_activations = np.memmap('neur.mymemmap', dtype='float32', mode='w+', shape=(datapoints * token_amount, d_model))
    batch_size = 32
    # dictionary_activations = np.zeros((datapoints*token_amount, num_features))
    smaller_auto_encoder = autoencoder
    smaller_auto_encoder.to_device(device)

    # print("Creating data dict")
    # d = DatasetDict(Dataset(dataset[ind_low:ind_max]))
    # print("Got data dict")
    # with torch.no_grad(), d.formatted_as("pt"):
    with torch.no_grad(), dataset.formatted_as("pt"):
        dl = DataLoader(dataset["input_ids"], batch_size=batch_size)
        for i, batch in enumerate(tqdm(dl)):
            # print(batch)
            _, cache = model.run_with_cache(batch.to(device))
            batched_neuron_activations = rearrange(cache[cache_name], "b s n -> (b s) n" )
            neuron_activations[i*batch_size*token_amount:(i+1)*batch_size*token_amount,:] = batched_neuron_activations.cpu()
            batched_dictionary_activations = smaller_auto_encoder.encode(batched_neuron_activations)
            dictionary_activations_final[i*batch_size*token_amount:(i+1)*batch_size*token_amount,:] = batched_dictionary_activations.cpu().numpy()
    del neuron_activations
        # dictionary_activations_final[ind_low:ind_max] = dictionary_activations
        # chunks.append(dictionary_activations)
    return dictionary_activations_final

dict_activations_main = get_activations(autoencoder_main)
dict_activations_prior = get_activations(autoencoder_prior)
# TODO: I think that we may want to save this to disk for the future


  0%|          | 0/3102 [00:00<?, ?it/s]

# Feature Interp
Investigate the example sentences the activate this feature.

Max: show max activating (tokens,contexts)

Uniform: Show range of activations from each bin (e.g. sample an example from 1-2, 2-3, etc). 
[Note: if a feature is monosemantic, then the full range of activations should be that feature, not just max-activating ones]

Full_text: shows the full text example

Text_list: shows up to the most activating example (try w/ max activating on a couple of examples to see)

ablate_text: remove the context one token at a time, and show the decrease/increase in activation of that feature

ablate_feature_direction: removes feature direction from model's activation mid-inference, showing the logit diff in the output for every token.

logit_lens: show the logit lens for that feature. If matches ablate_feature_direction, then the computation path is through the residual stream, else, it's through future layers

In [ ]:
dict_activations_prior.shape

torch.Size([39320, 3072])

In [ ]:
from circuitsvis.activations import text_neuron_activations
import torch
import numpy as np
from einops import rearrange

# Get the activations for the best dict features
def get_feature_datapoints_with_idx(feature_index, dictionary_activations, tokenizer, token_amount, dataset, k=10, setting="max"):
    best_feature_activations = dictionary_activations[:, feature_index]
    # Sort the features by activation, get the indices
    if setting=="max":
        found_indices = torch.argsort(best_feature_activations, descending=True)[:k]
    elif setting=="uniform":
        # min_value = torch.min(best_feature_activations)
        min_value = torch.min(best_feature_activations)
        max_value = torch.max(best_feature_activations)

        # Define the number of bins
        num_bins = k

        # Calculate the bin boundaries as linear interpolation between min and max
        bin_boundaries = torch.linspace(min_value, max_value, num_bins + 1)

        # Assign each activation to its respective bin
        bins = torch.bucketize(best_feature_activations, bin_boundaries)

        # Initialize a list to store the sampled indices
        sampled_indices = []

        # Sample from each bin
        for bin_idx in torch.unique(bins):
            if(bin_idx==0): # Skip the first one. This is below the median
                continue
            # Get the indices corresponding to the current bin
            bin_indices = torch.nonzero(bins == bin_idx, as_tuple=False).squeeze(dim=1)
            
            # Randomly sample from the current bin
            sampled_indices.extend(np.random.choice(bin_indices, size=1, replace=False))

        # Convert the sampled indices to a PyTorch tensor & reverse order
        found_indices = torch.tensor(sampled_indices).long().flip(dims=[0])
    else: # random
        # get nonzero indices
        nonzero_indices = torch.nonzero(best_feature_activations)[:, 0]
        # shuffle
        shuffled_indices = nonzero_indices[torch.randperm(nonzero_indices.shape[0])]
        found_indices = shuffled_indices[:k]
    num_datapoints = int(dictionary_activations.shape[0]/token_amount)
    datapoint_indices =[np.unravel_index(i, (num_datapoints, token_amount)) for i in found_indices]
    text_list = []
    full_text = []
    token_list = []
    full_token_list = []
    for md, s_ind in datapoint_indices:
        md = int(md)
        s_ind = int(s_ind)
        full_tok = torch.tensor(dataset[md]["input_ids"])
        full_text.append(tokenizer.decode(full_tok))
        tok = dataset[md]["input_ids"][:s_ind+1]
        text = tokenizer.decode(tok)
        text_list.append(text)
        token_list.append(tok)
        full_token_list.append(full_tok)
    return text_list, full_text, token_list, full_token_list, found_indices

In [ ]:
from interp_utils import *
# Can sort by MMCS w/ the larger dictionary
# indexed_feature = 13
# best_feature = int(max_indices[indexed_feature])
# Or just random index 
feature_idx = 13

text_list, full_text, token_list, full_token_list, sampled_indices = get_feature_datapoints_with_idx(feature_idx, dict_activations_prior, model.tokenizer, token_amount, dataset, setting="max")
# text_list, full_text, token_list, full_token_list = get_feature_datapoints(feature_idx, dict_activations_prior, model.tokenizer, token_amount, dataset, setting="max")
# text_list, full_text, token_list, full_token_list = get_feature_datapoints(best_feature, dictionary_activations, dataset, setting="max")
# visualize_text(full_text, best_feature, model, autoencoder, layer)
print("Sampled Index", sampled_indices)
visualize_text(text_list, feature_idx, model, autoencoder_prior, layer_focus - 1)

Sampled Index tensor([ 6738, 38178, 14329, 31726, 31759, 15716, 18798, 25789, 25622, 35054])


## Find the **downstream** feature in the next layer that is most activated by random subset of this feature

In [ ]:
prior_feature_idx = 12
k = 100

text_list, full_text, token_list, full_token_list, sampled_indices = get_feature_datapoints_with_idx(prior_feature_idx, dict_activations_prior, model.tokenizer, token_amount, dataset, setting="max", k=k)

# print("Sampled Index", sampled_indices)
# dict_activations_prior.shape, dict_activations_main.shape
TOP_FEATURES = 20
# We do not need absolute value as I think that everything is positive b/c ReLU
summed = dict_activations_main[sampled_indices].sum(dim=0)
summed_abs = summed.abs()
print(summed.argmax())
main_features = summed.argsort(descending=True)[:TOP_FEATURES]
main_features

tensor(1287)


tensor([1287, 1321, 2985,  621,  833, 2221, 1483, 1067,  189, 1926, 1104, 1002,
          12,  271,  663, 1414, 1515, 1112, 1343, 1101])

In [ ]:
visualize_text(text_list[:10], feature_idx, model, autoencoder_prior, layer_focus - 1)

In [ ]:
feature_idx = main_features[2].item()
 
k = 20
text_list, full_text, token_list, full_token_list, sampled_indices = get_feature_datapoints_with_idx(feature_idx, dict_activations_main, model.tokenizer, token_amount, dataset, setting="uniform", k=k)
visualize_text(text_list, feature_idx, model, autoencoder_prior, layer_focus)